In [1]:
import sqlite3
import urllib
import ssl 
from urlparse import urljoin
from urlparse import urlparse
from BeautifulSoup import *
scontext = None

In [2]:
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [3]:
cur.execute('''CREATE TABLE IF NOT EXISTS Pages 
    (id INTEGER PRIMARY KEY, url TEXT UNIQUE, html TEXT, 
     error INTEGER, old_rank REAL, new_rank REAL)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Links 
    (from_id INTEGER, to_id INTEGER)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Webs (url TEXT UNIQUE)''')

In [4]:
cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
row = cur.fetchone()
if row is not None:
    print "Restarting existing crawl."
else :
    starturl = raw_input('Enter web url or enter: ')
    if ( len(starturl) < 1 ) : starturl = 'http://python-data.dr-chuck.net/'
    if ( starturl.endswith('/') ) : starturl = starturl[:-1]
    web = starturl
    if ( starturl.endswith('.htm') or starturl.endswith('.html') ) :
        pos = starturl.rfind('/')
        web = starturl[:pos]

    if ( len(web) > 1 ) :
        cur.execute('INSERT OR IGNORE INTO Webs (url) VALUES ( ? )', ( web, ) )
        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( starturl, ) ) 
        conn.commit()

Restarting existing crawl.


In [ ]:
cur.execute('''SELECT url FROM Webs''')
webs = list()
for row in cur:
    webs.append(str(row[0]))

print webs

num = 0

['www.columbia.edu', 'http://python-data.dr-chuck.net', 'www.coursera.org']


In [ ]:
while True:
    if ( num < 1 ) :
        sval = raw_input('How many pages:')
        if ( len(sval) < 1 ) : break
        num = int(sval)
    num -= 1

    cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
    try:
        row = cur.fetchone()
        fromid = row[0]
        url = row[1]
    except:
        print 'No unretrieved HTML pages found'
        many = 0
        break

    print fromid, url, 

    # If we are retrieving this page, there should be no links from it
    cur.execute('DELETE from Links WHERE from_id=?', (fromid, ) )
    try:
        document = urllib.urlopen(url)

        html = document.read()
        if document.getcode() != 200 :
            print "Error on page: ",document.getcode()
            cur.execute('UPDATE Pages SET error=? WHERE url=?', (document.getcode(), url) )

        if 'text/html' != document.info().gettype() :
            print "Ignore non text/html page"
            cur.execute('UPDATE Pages SET error=-1 WHERE url=?', (url, ) )
            conn.commit()
            continue

        print '('+str(len(html))+')',

        soup = BeautifulSoup(html)
    except KeyboardInterrupt:
        print ''
        print 'Program interrupted by user...'
        break
    except:
        print "Unable to retrieve or parse page"
        cur.execute('UPDATE Pages SET error=-1 WHERE url=?', (url, ) )
        conn.commit()
        continue

    cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( url, ) ) 
    cur.execute('UPDATE Pages SET html=? WHERE url=?', (buffer(html), url ) )
    conn.commit()

    # Retrieve all of the anchor tags
    tags = soup('a')
    count = 0
    for tag in tags:
        href = tag.get('href', None)
        if ( href is None ) : continue
        # Resolve relative references like href="/contact"
        up = urlparse(href)
        if ( len(up.scheme) < 1 ) :
            href = urljoin(url, href)
        ipos = href.find('#')
        if ( ipos > 1 ) : href = href[:ipos]
        if ( href.endswith('.png') or href.endswith('.jpg') or href.endswith('.gif') ) : continue
        if ( href.endswith('/') ) : href = href[:-1]
        if ( len(href) < 1 ) : continue

        found = False
        for web in webs:
            if ( href.startswith(web) ) :
                found = True
                break
        if not found : continue

        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( href, ) ) 
        count = count + 1
        conn.commit()

        cur.execute('SELECT id FROM Pages WHERE url=? LIMIT 1', ( href, ))
        try:
            row = cur.fetchone()
            toid = row[0]
        except:
            print 'Could not retrieve id'
            continue
        cur.execute('INSERT OR IGNORE INTO Links (from_id, to_id) VALUES ( ?, ? )', ( fromid, toid ) ) 


    print count

cur.close()

How many pages:50
1267 http://python-data.dr-chuck.net/known_by_Karlie.html (12053) 100
1709 http://python-data.dr-chuck.net/known_by_Kainui.html (11990) 100
957 http://python-data.dr-chuck.net/known_by_Mahnoor.html (12050) 100
2808 http://python-data.dr-chuck.net/known_by_Simonne.html (12083) 100
2539 http://python-data.dr-chuck.net/known_by_Eshaal.html (12037) 100
2496 http://python-data.dr-chuck.net/known_by_Sorcha.html (12008) 100
349 http://python-data.dr-chuck.net/known_by_Jayse.html (12022) 100
2776 http://python-data.dr-chuck.net/known_by_Aaran.html (12046) 100
306 http://python-data.dr-chuck.net/known_by_Nazzera.html (12090) 100
2023 http://python-data.dr-chuck.net/known_by_Devon.html (12079) 100
2517 http://python-data.dr-chuck.net/known_by_Pearsen.html (12092) 100
1794 http://python-data.dr-chuck.net/known_by_Carter.html (11998) 100
1260 http://python-data.dr-chuck.net/known_by_Teejay.html (12099) 100
2143 http://python-data.dr-chuck.net/known_by_Machlan.html (12043) 100
182